caffe程序自带有一张小猫图片，存放路径为caffe根目录下的examples/images/cat.jpg，如果我们想用一个训练好的caffemodel来对这张图片进行分类，那该怎么办呢？如果不用这张小猫图片，换一张别的图片，又该怎么办？如果学会了小猫图片的分类，那么换成其他图片，程序实际上是一样的。

开发caffe的贾扬清团队，利用imagenet图片和caffenet模型训练好了一个caffemodel，供大家下载。要进行分类，这个caffemodel是最好不过的了。所以，不管是用C++进行分类，还是用python接口进行分类，我们都应该准备3个文件：

1、caffemodel文件

可以直接在浏览器输入地址下载，也可以运行脚本文件下载。[下载地址](http://dl.caffe.berkeleyvision.org/bvlc_reference_caffenet.caffemodel)

文件名称为：bvlc_reference_caffenet.caffemodel，文件大小为230M左右，为了代码的统一，将这个caffemodel文件下载到caffe根目录下的models/bvlc_reference_caffenet/文件夹下面。也可以运行脚本文件进行下载：

```shell
sh scripts/download_model_binary.py models/bvlc_reference_caffenet
```

2、均值文件

有了caffemodel文件，就需要对应的均值文件，在测试阶段，需要把测试数据减去均值。这个文件我们用脚本下载，在caffe根目录下执行：

```shell
sh data/ilsvrc12/get_ilsvrc_aux.sh
```

执行并下载后，均值文件放在data/ilsvrc12/文件夹下。

3、synset_words.txt文件

在调用脚本文件下载均值的时候，这个文件也一并下载好了。里面放的是1000个类的名称。

数据准备好了，我们就可以开始分类了，我们给大家提供两个版本的分类方法：

## 一、C++方法

在caffe根目录下的examples/cpp-classification/文件夹下，有个classification.cpp文件，就是用来分类的。当然编译后，放在build/examples/cpp_classification/下面。我们直接运行命令：

```shell
build/examples/cpp_classification/classification.bin \
models/bvlc_reference_caffenet/deploy.prototxt \
models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel \
data/ilsvrc12/imagenet_mean.binaryproto \
data/ilsvrc12/sysnet_words.txt \
examples/images/cat.jpg
```

命令很长，用了很多的'\'符号来换行。可以看出，从第2行开始就是参数，每行一个，共需要4个参数。

运行成功后，输出top-5结果：

```
---------- Prediction for examples/images/cat.jpg ----------
0.3134 - "n02123045 tabby, tabby cat"
0.2380 - "n02123159 tiger cat"
0.1235 - "n02124075 Egyptian cat"
0.1003 - "n02119022 red fox, Vulpes vulpes"
0.0715 - "n02127052 lynx, catamount"
```

即有0.3134的概率为tabby cat，有0.2380的概率为tiger cat，……

## 二、Python方法

Python接口可以使用jupyter notebook来进行可视化操作，因此推荐使用这种方法。

在这里我就不用可视化了，编写一个py文件，命名为py-classify.py：

In [ ]:
#coding=utf-8
#加载必要的库
import numpy as np

import sys,os

#设置当前目录
caffe_root = '/home/ubuntu/Downloads/caffe-master/' 
sys.path.insert(0, caffe_root + 'python')
import caffe
os.chdir(caffe_root)

net_file=caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
caffe_model=caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
mean_file=caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy'

net = caffe.Net(net_file,caffe_model,caffe.TEST)
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(mean_file).mean(1).mean(1))
transformer.set_raw_scale('data', 255) 
transformer.set_channel_swap('data', (2,1,0))

im=caffe.io.load_image(caffe_root+'examples/images/cat.jpg')
net.blobs['data'].data[...] = transformer.preprocess('data',im)
out = net.forward()


imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')

top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
for i in np.arange(top_k.size):
    print top_k[i], labels[top_k[i]]

执行这个文件，输出：

```
281 n02123045 tabby, tabby cat
282 n02123159 tiger cat
285 n02124075 Egyptian cat
277 n02119022 red fox, Vulpes vulpes
287 n02127052 lynx, catamount
```

caffe开发团队实际上也编写了一个python版本的分类文件，路径为python/classify.py。

运行这个文件必需两个参数，一个输入图片文件，一个输出结果文件。而且运行必须在python目录下。假设当前目录是caffe根目录，则运行：

```shell
cd python
python classify.py ../examples/images/cat.jpg result.npy
```

分类的结果保存在当前目录下的result.npy文件里面，是看不见的，而且这个文件有错误，运行的时候，会提示：

```
Mean shape incompatible with input shape
```

的错误。因此，要使用这个文件，我们还得进行修改：

1、修改均值计算：

定位到

```python
mean = np.load(args.mean_file)
```

这一行，在下面加上一行：

```python
mean = mean.mean(1).mean(1)
```

则可以解决报错的问题。

2、修改文件，使得结果显示在命令行下：

定位到

```python
#classify
start = time.time()
predictions = classifier.predict(inputs, not args.center_only)
print("Done in %.2f s." % (time.time() = start))
```

这个地方后面加上几行，如下所示：

```python
imagenet_labels_filename = '../data/ilsvrc12/synset_words.txt'
labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')
top_k = predictions.flatten().argsort()[-1:-6:-1]
for i in np.arange(top_k.size):
    print(top_k[i], labels[top_k[i]])
```

这样就可以了，运行不会报错，而且结果会显示在命令行下面。